## Introduction

This notebook will document the creation of a real estate price and information dataset. I will use packages BeautifulSoup and Selenium to scrape data from realestate.com.au and domain.com.au and then process it into a structured dataset for modelling.

In [26]:
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import sys
import numpy as np
import pandas as pd
import regex as re
import requests
from math import ceil
from time import sleep, time, perf_counter
from random import randint
from IPython.core.display import clear_output
from fake_useragent import UserAgent

### Testing
Open the Firefox browser and go to website using Selenium

In [218]:
# Open firefox browser
browser = webdriver.Firefox()

# Go to Realestate.com.au, NSW properties 1st page
browser.get('https://www.realestate.com.au/buy/in-nsw/list-1')

First we need to inspect the HTML of the site to look for the tags which store the links to the detailed property information pages. Let's use BeautifulSoup to scrape the page's HTML and telling it to find that details link for each property.

In [219]:
soup = BeautifulSoup(browser.page_source, 'html.parser')

# Get links to each house's webpage
listings = soup.find_all("a", class_="details-link residential-card__details-link")
listings[:5]

[<a class="details-link residential-card__details-link" href="/property-apartment-nsw-bondi-131687742"><span class="">11/28 Edward Street, Bondi</span></a>,
 <a class="details-link residential-card__details-link" href="/property-house-nsw-riverstone-131892442"><span class="">56 McCulloch Street, Riverstone</span></a>,
 <a class="details-link residential-card__details-link" href="/property-house-nsw-harrington+park-131784446"><span class="">5 Sir Warwick Fairfax Drive, Harrington Park</span></a>,
 <a class="details-link residential-card__details-link" href="/property-house-nsw-blacktown-131892406"><span class="">8 Kastelan Street, Blacktown</span></a>,
 <a class="details-link residential-card__details-link" href="/property-house-nsw-narrabeen-131783742"><span class="">11 Albemarle Street, Narrabeen</span></a>]

Each one of these entries contains a 'class' and a 'href' attribute. What we are after is the href tag, which gives us the suffix of the specific property's webpage link. To extract that attribute, very simply do the following:

In [55]:
listings[0]['href']

'/property-unit-nsw-westmead-131784298'

To get the full link, we just need to append the suffix to the root URL, https://www.realestate.com.au. 

In [56]:
property_links = [ 'https://www.realestate.com.au' + suffix['href'] for suffix in listings ]
property_links[:5]

['https://www.realestate.com.au/property-unit-nsw-westmead-131784298',
 'https://www.realestate.com.au/property-unit-nsw-manly+vale-131599398',
 'https://www.realestate.com.au/property-unit-nsw-albury-131892214',
 'https://www.realestate.com.au/property-house-nsw-rozelle-131784258',
 'https://www.realestate.com.au/property-apartment-nsw-coolangatta-131892246']

Now that we have all the property links to the first page, we need to replicate the same process for the 2nd, 3rd, 4th etc. pages in the RealEstate website (just for NSW there are 55000 homes).

FOr example the below snippet is the html corresponding to the 'Next' button (we can see it is just 'list-2', 'list-3' etc, so perhaps we could even just hardcode this in):

```HTML
<a href="/buy/property-house-in-nsw/list-2?includeSurrounding=false" class="rui-button-brand pagination__link-next" title="Go to Next Page" rel="next"><span class="pagination__next-label">Next</span><span class="rui-icon rui-icon-forward-small"></span></a>
```

Again, we can use BeautifulSoup's find_all method to look for 'a' tags can the class "rui-button-brand pagination__link-next".

In [220]:
# Get link to next page
nextpage = soup.find_all("a", class_="rui-button-brand pagination__link-next")
nextpage_link = 'https://www.realestate.com.au' + nextpage[0]['href']
nextpage_link

'https://www.realestate.com.au/buy/in-nsw/list-2'

We actually don't need to use Selenium to scrape realestate.com.au, since its mostly a static html webpage. I can even just loop through the page numbers (ie. list-1,list-2,list-3...). Instead of Selenium I will use the python requests library to get the website.

Below is the same snippet fom above that extracts the desired html tags from the site.

Note: must specify user-agent***

In [4]:
headers={'User-Agent': "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36"}
req  = requests.get('https://www.realestate.com.au/buy/in-2155/list-1', headers=headers)
soup = BeautifulSoup(req.content, 'html.parser')

# Get links to each house's webpage
links = soup.find_all("a", class_="details-link residential-card__details-link")
links[0]['href']

[<a class="details-link residential-card__details-link" href="/property-apartment-nsw-rouse+hill-131704398"><span class="">66/97 Caddies Boulevard, Rouse Hill</span></a>,
 <a class="details-link residential-card__details-link" href="/property-house-nsw-north+kellyville-131803638"><span class="">10 Blue Wren Way, North Kellyville</span></a>,
 <a class="details-link residential-card__details-link" href="/property-apartment-nsw-rouse+hill-131913202"><span class="">5/93 Caddies Boulevard, Rouse Hill</span></a>,
 <a class="details-link residential-card__details-link" href="/property-house-nsw-kellyville-131911202"><span class="">10 Blackham Road, Kellyville</span></a>,
 <a class="details-link residential-card__details-link" href="/property-house-nsw-kellyville-131910858"><span class="">35 Glenrowan Avenue, Kellyville</span></a>]

### Scraping Code

We need a some code to better handle retries in the web scrape since errors can happen at any point. The code ideally should also be able to retain the list obtained so far and save the page number and postcode that the web crawler is up to.

In [27]:
# This code comes from: https://www.peterbe.com/plog/best-practice-with-retries-with-requests
# Basically we 'replace requests.get(...)', with 'requests_retry_session().get(...)'
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry


def requests_retry_session(
    retries=3,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

In [28]:
def get_property_links(url, max_pages,headers):
    property_links = []
    
    for i in range(max_pages):
        req  = requests_retry_session().get( url + str(i+1) + '?includeSurrounding=false' ,headers=headers)
        soup = BeautifulSoup(req.content, 'html.parser')
        listings = soup.find_all("a", class_="details-link residential-card__details-link")
        page_link = ['https://www.realestate.com.au'+row['href'] for row in listings]
        property_links.extend(page_link)
        print('page'+str(i+1))
        sleep(np.random.lognormal(0,1))
    
    return property_links

In [4]:
headers={'User-Agent': "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36"}

start=perf_counter()
property_links = get_property_links('https://www.realestate.com.au/buy/in-2000/list-', 3,headers)
end=perf_counter()
end-start

page1
page2
page3


19.8397153

In [5]:
property_links[:5]

['https://www.realestate.com.au/property-apartment-nsw-haymarket-131628150',
 'https://www.realestate.com.au/property-apartment-nsw-sydney-131625954',
 'https://www.realestate.com.au/property-apartment-nsw-sydney-131821082',
 'https://www.realestate.com.au/property-apartment-nsw-sydney-131813602',
 'https://www.realestate.com.au/property-apartment-nsw-sydney-131713782']

One thing to make sure is the crawler doesn't try to go beyond the last page. If it does, it either starts looking at surrounding suburbs, or just displays a blank page. We can use the number of results summary at the top of the list (eg. '25 of 48 results') to calculate the maximum number of pages to visit.

In [6]:
req_test  = requests_retry_session().get( 'https://www.realestate.com.au/buy/in-pyrmont/list-1?includeSurrounding=false' ,headers=headers)
soup = BeautifulSoup(req_test.content, 'html.parser')
html_section = soup.find("div", class_="results-set-header__summary")
print(html_section.text)
print(re.findall(r"(\d+) result", html_section.text)[0])

1-25 of 51 results
51


In [29]:
def get_max_pages(url,useragent):
    headers = {'User-Agent': str(useragent)}
    req  = requests_retry_session().get( url + '1?includeSurrounding=false' ,headers=headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    results = soup.find("div", class_="results-set-header__summary")
    num_results = re.findall(r"(\d+) result", results.text)[0]
    max_pages = ceil(int(num_results)/25)
    return max_pages

Realestate.com.au does not display all 55000 homes when I filter by NSW, it limits the number of listed pages to 80. Therefore, it only shows up to 2000 homes, which is not a large enough dataset for us. One thing we can do is get a full NSW postcode list and do a search for each postcode (most likely no postcodes have more than 2000 ads). I found a postcode list that's been generously made public by Matthew Proctor here: https://www.matthewproctor.com/australian_postcodes

After testing manually, looks like when you search by postcode the website becomes: 

URL = 'https://www.realestate.com.au/buy/in' + POSTCODE + '/list-' + PAGE_NUMBER

Below is code to loop through loop through each postcode and then extract each link.

In [30]:
# Get Postcodes
postcodes = pd.read_csv("./data/australian_postcodes.csv")
postcode_list=postcodes[postcodes.type=="Delivery Area"].postcode
nsw_postcodes = postcode_list[(postcode_list>=2000) & (postcode_list<=2999)].unique()
nsw_postcodes[:20]

array([2000, 2006, 2007, 2008, 2009, 2010, 2011, 2015, 2016, 2017, 2018,
       2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027], dtype=int64)

### Get All Property Links
For each postcode, get the max number of pages from the results count, and then loop through to get all the property links. 

In [9]:
nsw_property_links = []
num_requests = 0
start_time = time()
i=0
headers={'User-Agent': "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36"}


for postcode in nsw_postcodes:
    url = 'https://www.realestate.com.au/buy/in-' + str(postcode) + '/list-'
    max_pages = get_max_pages(url, headers=headers) # get the number of pages of results
    num_requests += 1
    print('Postcode: '+ str(postcode) + ', Total pages: ' + str(max_pages))
    
    if max_pages == 0: # if there are 0 results, skip to the next postcode
        continue
    
    sleep(np.random.lognormal(0.3,1))
    
    nsw_property_links.extend(get_property_links(url, max_pages,headers)) # get all property links from each page, and add to end of list
    num_requests += max_pages
    elapsed_time = time() - start_time
    print('Request: {}; Frequency: {} requests/s'.format(num_requests, num_requests/elapsed_time))
    
    i+=1
    if i>=5:
        clear_output(wait=True)
        i=0
    

Postcode: 2911, Total pages: 1
page1
Request: 3260; Frequency: 0.40930754752330795 requests/s
Postcode: 2912, Total pages: 3
page1
page2
page3
Request: 3264; Frequency: 0.4091082794707579 requests/s
Postcode: 2913, Total pages: 7
page1
page2
page3
page4
page5
page6
page7
Request: 3272; Frequency: 0.40815987554358024 requests/s
Postcode: 2914, Total pages: 9
page1
page2
page3
page4
page5
page6
page7
page8
page9
Request: 3282; Frequency: 0.4079151073438699 requests/s


Yay! Looks like we have all 57k property links downloaded without errors! Now we can save the list of links using pickle

In [45]:
import pickle
# with open("./data/nsw_property_links.txt", "wb") as fp:   #Pickling
#     pickle.dump(nsw_property_links, fp)

In [18]:
import pickle

with open("./data/nsw_property_links.txt", "rb") as fp:   # Unpickling
    nsw_property_links = pickle.load(fp)

### Get Property Data Using Only Listing Pages
Let's try to get as much property data as possible from the listing pages, since that's only 2200 requests as opposed to the 55000 requests if we extract from each individual property's web page.

In [31]:
# Functions to extract features from Listing Pages (each page with approx 25 properties)
def get_link(soup):
    try:
        link = 'https://www.realestate.com.au' +  soup.find("a", class_="details-link residential-card__details-link")["href"]
        return link
    except:
        return "None"

def get_address(soup):
    try:
        address = soup.find("h2", class_="residential-card__address-heading").text
        return address
    except:
        return "None"

def get_price(soup):
    try:
        price =  soup.find("span", class_="property-price").text
        return price
    except:
        return "None"
    
def get_beds(soup):
    try:
        beds = soup.find("span", class_="general-features__icon general-features__beds").text
        return beds
    except:
        return "None"

def get_baths(soup):
    try:
        baths = soup.find("span", class_="general-features__icon general-features__baths").text
        return baths
    except:
        return "None"
    
def get_cars(soup):
    try:
        cars = soup.find("span", class_="general-features__icon general-features__cars").text
        return cars
    except:
        return "None"

def get_property_area(soup):
    try:
        property_area = soup.find("span", class_="property-size__icon property-size__building").text
        return property_area
    except:
        return "None"
    
def get_land_area(soup):
    try:
        land_area = soup.find("span", class_="property-size__icon property-size__land").text
        return land_area
    except:
        return "None"
    
def get_property_type(soup):
    try:
        property_type = soup.find("span", class_="residential-card__property-type").text
        return property_type
    except:
        return "None"

In [96]:
def get_html_data(url,useragent):
    headers={'User-Agent': str(useragent)}
    req  = requests_retry_session().get( url, headers=headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    return soup

def get_property_data(url,postcode, max_pages,useragent):
    property_data = []
    
    for i in range(max_pages):
        
        soup = get_html_data(url + str(i+1) + '?includeSurrounding=false', useragent)
        
        all_listings_html = soup.find_all("div", {'class':'residential-card__content'})
        for listing in all_listings_html:
            link = get_link(listing)
            address = get_address(listing)
            price = get_price(listing)
            beds = get_beds(listing)
            baths = get_baths(listing)
            cars = get_cars(listing)
            property_area = get_property_area(listing)
            land_area = get_land_area(listing)
            property_type = get_property_type(listing)
            property_data.append([link,
                      address,
                      postcode,            
                      price,
                      beds,
                      baths,
                      cars,
                      property_area,
                      land_area,
                      property_type])  
        
        print('page'+str(i+1))
        sleep(np.random.lognormal(0,1))
        
    return property_data

Similar code to ###'Get All Property Links' above, but we will use the get_property_data function to get the other features which are listed on the listings pages.

In [100]:
nsw_property_data = []
num_requests = 0
i=0
ua = UserAgent() # Get list of fake user agents

start_time = time()
for postcode in nsw_postcodes:
    url = 'https://www.realestate.com.au/buy/in-' + str(postcode) + '/list-'
    max_pages = get_max_pages(url, useragent=ua.random) # get the number of pages of results
    num_requests += 1
    print('Postcode: '+ str(postcode) + ', Total pages: ' + str(max_pages))
    
    if max_pages == 0: # if there are 0 results, skip to the next postcode
        continue
    
    sleep(np.random.lognormal(0,1))
    
    nsw_property_data.extend(get_property_data(url,postcode, max_pages, useragent=ua.random)) # get all property links from each page, and add to end of list
    num_requests += max_pages
    elapsed_time = time() - start_time
    print('Requests: {}; Frequency: {:.3f} requests/s'.format(num_requests, num_requests/elapsed_time))
    print('Total Properties: {}; Elapsed Time: {:.3f}'.format(len(nsw_property_data),elapsed_time))
    
    i+=1
    if i>=5:
        clear_output(wait=True)
        i=0
    

Postcode: 2911, Total pages: 1
page1
Requests: 3250; Frequency: 0.437 requests/s
Total Properties: 57254; Elapsed Time: 7432.739
Postcode: 2912, Total pages: 3
page1
page2
page3
Requests: 3254; Frequency: 0.437 requests/s
Total Properties: 57321; Elapsed Time: 7444.147
Postcode: 2913, Total pages: 7
page1
page2
page3
page4
page5
page6
page7
Requests: 3262; Frequency: 0.436 requests/s
Total Properties: 57479; Elapsed Time: 7489.243
Postcode: 2914, Total pages: 9
page1
page2
page3
page4
page5
page6
page7
page8
page9
Requests: 3272; Frequency: 0.436 requests/s
Total Properties: 57680; Elapsed Time: 7509.152


In [106]:
len(nsw_property_data)

57680

In [102]:
nsw_property_data[:10]

[['https://www.realestate.com.au/property-apartment-nsw-sydney-131924626',
  '511/188 Day Street, Sydney',
  2000,
  'For Sale | Obsidian Property',
  ' 3',
  ' 2',
  ' 1',
  'None',
  'None',
  'Apartment'],
 ['https://www.realestate.com.au/property-apartment-nsw-haymarket-131628150',
  '1803/178 Thomas Street, Haymarket',
  2000,
  'Contact Agent',
  ' 2',
  ' 2',
  ' 2',
  'None',
  'None',
  'Apartment'],
 ['https://www.realestate.com.au/property-apartment-nsw-sydney-131625954',
  '2406/168 Kent Street, Sydney',
  2000,
  '$1,540,000',
  ' 1',
  ' 1',
  ' 1',
  'None',
  'None',
  'Apartment'],
 ['https://www.realestate.com.au/property-apartment-nsw-sydney-131821082',
  '806/1 Hosking Place, Sydney',
  2000,
  '$719,000',
  ' 1',
  ' 1',
  'None',
  'None',
  'None',
  'Apartment'],
 ['https://www.realestate.com.au/property-apartment-nsw-sydney-131813602',
  '1702/11-15 Alberta Street, Sydney',
  2000,
  'Auction',
  ' 2',
  ' 2',
  ' 1',
  'None',
  'None',
  'Apartment'],
 ['http

In [104]:
# Full property data features
import pickle
with open("./data/nsw_property_data_full.pickle", "wb") as fp:   #Pickling
    pickle.dump(nsw_property_data, fp)

In [105]:
import time
file_name = "data/nsw_property_data_" + str(time.strftime("%Y-%m-%d")) + ".csv"

columns = ["link",
           "address", 
           "postcode", # If I run this next time, I will include the postcode
           "price", 
           "num_beds",
           "num_baths",
           "num_cars",
           "property_area",
           "land_area",
           "property_type"]
           
(pd.DataFrame(nsw_property_data, columns = columns).
    to_csv(file_name, index = False, encoding = "UTF-8"))


Let's fix a few things up, like leaving out the postcode. I can still match it back with the postcode mapping csv I had before.

In [39]:
nsw_property_data = pd.read_csv("data/nsw_property_data_2019-09-09.csv")

In [81]:
# Postcode-SUburb Mapping
postcode_mapping = pd.read_csv("./data/australian_postcodes.csv")
postcode_mapping_nsw = postcode_mapping.query("type=='Delivery Area' & postcode>=2000 & postcode<=2999")
postcode_mapping_nsw["locality"] = postcode_mapping_nsw["locality"].str.lower()
postcode_mapping_nsw.head()

C:\Users\martin.cheung\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,postcode,locality,state,long,lat,dc,type
568,2000,barangaroo,NSW,151.201580,151.201580,NaN,Delivery Area
570,2000,dawes point,NSW,151.256649,-33.859953,WATERLOO DELIVERY FACILITY,Delivery Area
571,2000,haymarket,NSW,151.256649,-33.859953,WATERLOO DELIVERY FACILITY,Delivery Area
572,2000,millers point,NSW,151.256649,-33.859953,WATERLOO DELIVERY FACILITY,Delivery Area
573,2000,parliament house,NSW,151.256649,-33.859953,WATERLOO DELIVERY FACILITY,Delivery Area


### Get Individual Property Features

In each individual webpage, now we have to extract the relevant features. Test out the tags to search for each property attribute below.

In [36]:
def get_html_data(url,useragent):
    headers={'User-Agent': str(useragent)}
    req  = requests_retry_session().get( url, headers=headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    return soup

Below is are functions to extract features from the individual web page. Note: the tags might be slightly different to the ones used to extract data from the listing pages

In [92]:
def get_address(soup):
    try:
        address = soup.find("h1", class_="property-info-address").text
        return address
    except:
        return "None"

def get_price(soup):
    try:
        price =  soup.find("span", class_="property-price property-info__price").text
        return price
    except:
        return "None"
    
def get_beds(soup):
    try:
        beds = soup.find("span", class_="general-features__icon general-features__beds").text
        return beds
    except:
        return "None"

def get_baths(soup):
    try:
        baths = soup.find("span", class_="general-features__icon general-features__baths").text
        return baths
    except:
        return "None"
    
def get_cars(soup):
    try:
        cars = soup.find("span", class_="general-features__icon general-features__cars").text
        return cars
    except:
        return "None"

def get_property_area(soup):
    try:
        property_area = soup.find("span", class_="property-size__icon property-size__building").text
        return property_area
    except:
        return "None"
    
def get_land_area(soup):
    try:
        land_area = soup.find("span", class_="property-size__icon property-size__land").text
        return land_area
    except:
        return "None"
    
def get_description(soup):
    try:
        description = soup.find("article", class_="property-description").text
        return description
    except:
        return "None"
    
def get_property_features(soup):
    try:
        property_features = soup.find_all("div", class_="property-features__feature")
        
        feature_list = [feature.text for feature in property_features]
        
        return feature_list
    except:
        return "None"

In [38]:
# Don't use free Proxies - can be risky since we don't know what malware and viruses are on there
# import requests
# from itertools import cycle
# import traceback

# with open('data/ProxyList.txt') as f:
#     text = f.readlines()
    
# proxies = [line.strip('\n') for line in text]

# proxy_pool = cycle(proxies)

In [66]:
def get_property_data(nsw_property_links):
    property_data = []
    i=0
    num_requests=0
    
    # Get list of fake user agents
    ua = UserAgent()
    
    start_time = time()
    for url in nsw_property_links:
        try:
            soup = get_html_data(url,ua.random)
        except:
            print("URL doesn't work. Try next one")
            continue
        
        address = get_address(soup)
        price = get_price(soup)
        beds = get_beds(soup)
        baths = get_baths(soup)
        cars = get_cars(soup)
        property_area = get_property_area(soup)
        land_area = get_land_area(soup)
        description = get_description(soup) 
        feature_list = get_property_features(soup)
        
        property_data.append([address,
                             price,
                             beds,
                             baths,
                             cars,
                             property_area,
                             land_area,
                             description,
                             feature_list])
        
        num_requests+=1

        sleep(np.random.lognormal(0.3,1)) # perhaps we don't need to sleep since using proxies is pretty slow already
        
        elapsed_time = time() - start_time
        print('Requests: {}; Elapsed Time: {:.3f}; Frequency: {:.3f} requests/s'.format(num_requests, elapsed_time, num_requests/elapsed_time))
        
        i+=1
        if i>=5:
            clear_output(wait=True)
            i=0
            
        if num_requests % 2000 == 0:
            with open("./data/nsw_property_data.pickle", "wb") as fp:   #Pickling
                pickle.dump(property_data, fp)
            
            print("overwrite pickle at request no.:", num_requests)
        
    return property_data

In [67]:
property_data_10k = get_property_data(nsw_property_links[:10000])

Property Number: 6266


KeyboardInterrupt: 

In [68]:
len(property_data_10k)

NameError: name 'property_data_10k' is not defined

In [46]:
property_data

[['1702/11-15 Alberta Street, Sydney, NSW 2000',
  'Auction',
  ' 2',
  ' 2',
  ' 1',
  'None',
  'None',
  'Contemporary urban sanctuary within footsteps of Hyde Park1702/11-15 Alberta Street, SydneyPromising lifestyle excellence in the heart of the city, this sleek apartment provides an idyllic urban sanctuary of contemporary style and space. Cleverly designed and brimming with natural light, it features a generous open layout with zoned living/dining areas plus a CaesarStone island kitchen with quality Smeg gas appliances. The bedrooms are well proportioned and appointed with built-in wardrobes, while sliding glass doors open to a full width undercover balcony and elevated city outlooks. Complete with secure parking, this superb offering is positioned within a stroll of Hyde Park and the CBD and enjoys shops and cafes at the doorstep.•\t2 bedroom, 2 bathroom, 1 car•\tModern security building by Marchese Partners architects•\tSleek contemporary design with zoned living/dining areas•\

In [47]:
with open("./data/nsw_property_data.pickle", "wb") as fp:   #Pickling
            pickle.dump(property_data, fp)

In [69]:
with open("./data/nsw_property_data.pickle", "rb") as fp:   #Pickling
            property_data=pickle.load(fp)